# Primer punto

Se tiene un robot manipulador cuyos valores articulares están dados por el vector $q = (q_1, q_2, q_3, q_4)$. Este robot posee cuatro grados de libertad y se encuentra modelado cinemáticamente a través de los parámetros de Denavit-Hartenberg estándar mostrados en la siguiente tabla, donde $L_1 = 0.6, L_2 = 0.8 y L_3 = 1$

| $i$ | $\theta_i$ | $d_i$ | $a_i$ | $\alpha_i$ |
|---|---|---|---|---|
| 1 | $q_1$ | 0 | $L_1$ | 0 |
| 2 | 0 | $q_2$ | 0 | $-\frac{\pi}{2}$ |
| 3 | $q_3$ | 0 | $L_2$ | $\frac{\pi}{2}$ |
| 4 | $q_4$ | $L_3$ | 0 | 0 |


**Tip: Matriz \( A_i \):**
\[
A_i = 
\begin{bmatrix}
\cos(\theta_i) & -\sin(\theta_i)\cos(\alpha_i) & \sin(\theta_i)\sin(\alpha_i) & a_i\cos(\theta_i) \\
\sin(\theta_i) & \cos(\theta_i)\cos(\alpha_i) & -\cos(\theta_i)\sin(\alpha_i) & a_i\sin(\theta_i) \\
0 & \sin(\alpha_i) & \cos(\alpha_i) & d_i \\
0 & 0 & 0 & 1
\end{bmatrix}
\]

In [11]:
import numpy as np

def get_end_effector_pose(q, L):
    # Denavit-Hartenberg parameters
    theta = [q[0], 0, q[2], q[3]]
    d = [0, q[1], 0, L[2]]
    a = [L[0], 0, L[1], 0]
    alpha = [0, -np.pi/2, np.pi/2, 0]

    # Initialize transformation matrix
    T = np.eye(4)

    # Iterate over each joint
    for i in range(4):
        # Compute transformation matrix for this joint
        A = np.array([
            [np.cos(theta[i]), -np.sin(theta[i])*np.cos(alpha[i]), np.sin(theta[i])*np.sin(alpha[i]), a[i]*np.cos(theta[i])],
            [np.sin(theta[i]), np.cos(theta[i])*np.cos(alpha[i]), -np.cos(theta[i])*np.sin(alpha[i]), a[i]*np.sin(theta[i])],
            [0, np.sin(alpha[i]), np.cos(alpha[i]), d[i]],
            [0, 0, 0, 1]
        ])

        # Update transformation matrix
        T = np.dot(T, A)

    return T

Los resultados para la posición y orientación que tendrá el efector final de este robot cuando el vector articular sea $ q = (1, 0, -\frac{\pi}{2}, \frac{\pi}{2}) $ es:

In [12]:
q = [1, 0, -np.pi/2, np.pi/2]  # joint angles
L = [0.6, 0.8, 1.0]  # link lengths
T = get_end_effector_pose(q, L)
print(T)

[[-8.41470985e-01 -1.36134449e-16 -5.40302306e-01 -2.16120922e-01]
 [ 5.40302306e-01  1.46427115e-17 -8.41470985e-01 -3.36588394e-01]
 [ 1.22464680e-16 -1.00000000e+00  6.12323400e-17  8.00000000e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


## Solución Robot Sawyer



In [13]:
import roboticstoolbox as rtb
import numpy as np

def create_custom_robot(dh_parameters):
    links = [rtb.RevoluteDH(d=d, a=a, alpha=alpha, offset=offset) for offset, d, a, alpha in dh_parameters]
    return rtb.DHRobot(links, name="Sawyer")

In [14]:
sawyer_dh_parameters = [
    (0, .317, -.081, -np.pi/2),
    (-np.pi/2, .192, 0, np.pi/2),
    (0, .4, 0, np.pi/2),
    (0, .168, 0, -np.pi/2),
    (0, .4, 0, -np.pi/2),
    (0, .1363, 0, np.pi/2),
    (0, .1335, 0, 0)
]

customized_robot = create_custom_robot(sawyer_dh_parameters)
customized_robot

DHRobot: Sawyer, 7 joints (RRRRRRR), dynamics, standard DH parameters
┌──────────┬────────┬────────┬────────┐
│   θⱼ     │   dⱼ   │   aⱼ   │   ⍺ⱼ   │
├──────────┼────────┼────────┼────────┤
│ q1       │  0.317 │ -0.081 │ -90.0° │
│ q2 - 90° │  0.192 │      0 │  90.0° │
│ q3       │    0.4 │      0 │  90.0° │
│ q4       │  0.168 │      0 │ -90.0° │
│ q5       │    0.4 │      0 │ -90.0° │
│ q6       │ 0.1363 │      0 │  90.0° │
│ q7       │ 0.1335 │      0 │   0.0° │
└──────────┴────────┴────────┴────────┘

┌─┬──┐
└─┴──┘

In [15]:
T = customized_robot.fkine([0,0,0,0,0,0,0])
print(T)

   0         0        -1        -1.014     
   0         1         0         0.1603    
   1         0         0         0.317     
   0         0         0         1         



# Students validation

In [19]:
sawyer_dh_parameters = [
    (0, .317, .081, np.pi/2),
    (np.pi/2, .140, 0.192, np.pi/2),
    (0, .260, 0.1685, -np.pi/2),
    (0, 0.1265, 0, np.pi/2),
    (0, .2735, 0, -np.pi/2),
    (0, .0, 0.0, np.pi/2),
    (0, 0, 0, 0)
]

customized_robot = create_custom_robot(sawyer_dh_parameters)
customized_robot

DHRobot: Sawyer, 7 joints (RRRRRRR), dynamics, standard DH parameters
┌──────────┬────────┬────────┬────────┐
│   θⱼ     │   dⱼ   │   aⱼ   │   ⍺ⱼ   │
├──────────┼────────┼────────┼────────┤
│ q1       │  0.317 │  0.081 │  90.0° │
│ q2 + 90° │   0.14 │  0.192 │  90.0° │
│ q3       │   0.26 │ 0.1685 │ -90.0° │
│ q4       │ 0.1265 │      0 │  90.0° │
│ q5       │ 0.2735 │      0 │ -90.0° │
│ q6       │      0 │      0 │  90.0° │
│ q7       │      0 │      0 │   0.0° │
└──────────┴────────┴────────┴────────┘

┌─┬──┐
└─┴──┘

In [20]:
T = customized_robot.fkine([0,0,0,0,0,0,0])
print(T)

   0         0         1         0.6145    
   0        -1         0        -0.2665    
   1         0         0         0.6775    
   0         0         0         1         

